# Validating Parameter Input

When writing scripts, ensuring that users provide valid input helps prevent errors and makes the script more reliable. PowerShell provides several parameter validation attributes that automatically check user input before the script executes. These attributes improve user experience and reduce the need for manual error handling inside your script. These validation methods are defined inline with the parameter.

## ValidateSet()
The ```[ValidateSet()]``` attribute restricts a parameter to only allow specific predefined values. If a user tries to pass a value outside this set, PowerShell will throw an error.

### Example: Limiting Choices
The following example has a parameter ```$Action``` that only accepts three different values: ```Start```, ```Stop```, and ```Restart```.

```powershell
param (
    [Parameter(Mandatory=$true)]
    [ValidateSet("Start", "Stop", "Restart")]
    [string]$Action
)

Write-Output "Performing action: $Action"
```

If the user tries to enter a value other than one defined in the set, they will receive an error message similar to this:

```Cannot validate argument on parameter 'Action'. The argument "Pause" does not belong to the set "Start,Stop,Restart" specified by the ValidateSet attribute.```

## ValidateRange()

The ```[ValidateRange()]``` attribute ensures that numeric inputs fall within a specific range. This method is useful for restricting values like percentages, time durations, or resource limits.

### Example: Enforcing a Number Range

The following example has a parameter ```$Percentage``` that only accepts values from 1 to 100.

```powershell
param (
    [ValidateRange(1, 100)]
    [int]$Percentage
)

Write-Output "Processing $Percentage%..."
```

## ValidatePattern()

The ```[ValidatePattern()]``` attribute uses regular expressions (**RegEx**) to enforce specific input formats. Validating patterns using RegEx is useful for inputs that follow well-known formats like email addresses, phone numbers, or file paths.

**Note**: RegEx can be a complex topic and is not covered here in depth.

The example below detects if a North American phone number. These numbers typically have a format like this:

- ```123-456-7890```

The regular expression ensures the phone number:
- Contains exactly 10 digits
- Is formatted with dashes (```-```)
- Uses only digits and dashes

```powershell
param (
    [ValidatePattern("^\d{3}-\d{3}-\d{4}$")]
    [string]$PhoneNumber
)
```

Here are the components of the regex pattern ```^\d{3}-\d{3}-\d{4}$```:
- ```^``` → Start of the string
- ```\d{3}``` → Exactly three digits
- ```-``` → A required dash (-)
- ```\d{3}``` → Exactly three more digits
- ```-``` → Another required dash (-)
- ```\d{4}``` → Exactly four digits
- ```$``` → End of the string

## ValidateLength()
The ```[ValidateLength()]``` attribute ensures that a string is within a specific length range.

For example, you have script for creating a Windows-based system, which has a 15-character limit. However, you also want the computer name to be at elast 5 characters.

The example parameter definition for ```$ComputerName``` uses ```[ValidateLength()]``` to verify the parameter value matches these requirements.

```powershell
param (
    [ValidateLength(5, 15)]
    [string]$ComputerName
)
```

## ValidateScript
If none of the built-in validation methods meet your needs, you can write a custom script and test the parameter value using ```[ValidateScript()]```. This method is useful for checking file paths, URLs, or database records prior to running a script.

The example below has a parameter named ```$FilePath``` that uses ```[ValidateScript()]``` to verify the path is valid prior to running the script. The use of ```$_``` in the script represents the parameter value being passed to the validation script.

```powershell
param (
    [ValidateScript({ Test-Path $_ })]
    [string]$FilePath
)
```

The script should return ```$true``` or perform an operation that results in a boolean value (e.g. ```Test-Path```). If the script determines the value is not valid, have the script return ```$false``` or throw an error.

However, [documentation](https://learn.microsoft.com/powershell/module/microsoft.powershell.core/about/about_throw#using-throw-to-create-a-mandatory-parameter) says not to use the ```throw``` keyword but instead return ```$false```. This note may be a reference to older PowerShell versions where using ```throw``` with a descriptive error message.

The ```ErrorMessage``` argument was added in PowerShell 6 ([reference](https://learn.microsoft.com/powershell/scripting/developer/cmdlet/validatescript-attribute-declaration#syntax)). An example of using ```ErrorMessage``` is shown below where ```{0}``` represents the parameter value.

```powershell
param (
    [ValidateScript({
        Test-Path $_ 
    }, ErrorMessage = "The file path '{0}' is not valid or does not exist.")]
    [string]$FilePath
)
```

If you are sure your scripts will be used in PowerShell 6 or higher, return ```$false``` and use the ```ErrorMessage``` argument. Otherwise, you can still use ```throw```.

## Validate for Not Null or Empty Strings
You can also validate that the parameter value is not ```$null``` or an empty string using ```[ValidateNotNull()]``` and ```[ValidateNotNullOrEmpty()]```. Essentially, you do not care what the value is, as long as the user provided a value.

- ```[ValidateNotNull()]``` – Prevents ```$null``` values but allows empty strings.
- ```[ValidateNotNullOrEmpty()]``` – Prevents both ```$null``` values and empty strings.

This example allows an empty string (```""```) but not ```$null```.

```powershell
param (
    [ValidateNotNull()]
    [string]$UserName
)
```
This example allows an empty string (```""```) *or* ```$null```.

```powershell
param (
    [ValidateNotNullOrEmpty()]
    [string]$UserName
)
```

## Exercise: Create a New User Script
Write a script for creating a new user with the following parameters and validation:

- A username (5-12 characters)
- Password expiration (30-90 days)
- Email address (hint: find a regex pattern to match an email address)

Display a message confirming the user's account information (*don't actually create any kind of account!*).

Test your script with different inputs to verify the parameter validation.

When ready, take a look at this example answer: [CreateUser.ps1](./solutions/CreateUser.ps1)